In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from pyod.models.pca import PCA
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize

%matplotlib inline

In [3]:
pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_excel('../../../OldDatasets/ML_data.xlsx')
#df.head()

In [7]:
df.dropna(axis=1, inplace=True)
#df.head()

In [8]:
descriptors = df.columns[10:]
descriptors

Index(['Z_A', 'Z_B', 'PBE_delta_H', 'PBE_latt_const', 'PBE_gap', 'Eps_elec',
       'Eps_ion', 'CM1', 'CM2', 'CM3', 'CM4', 'CM5', 'CM6', 'CM7', 'CM8',
       'Ion_rad', 'BP', 'MP', 'Density', 'At_wt', 'ICSD_vol', 'Cov_rad',
       'Ion_Energy', 'At_rad_1', 'Elec_Aff', 'At_rad_2', 'At_vol', 'Mend_num',
       'Ion_pot_1', 'Ion_pot_2', 'Thermal_expn', 'Sp_heat_cap', 'Therm_cond',
       'Elec_cond', 'Heat_fusion', 'Heat_vap', 'Electronegativity', 'At_num',
       'Period', 'Group', 'Valence', 'Ox_state'],
      dtype='object')

In [9]:
output = df.columns[4:10]
output

Index(['∆H (A-rich)', '∆H (B-rich)', '(+2/+1)', '(+1/0)', '(0/-1)', '(-1/-2)'], dtype='object')

In [10]:
df_nooutliers = df[(df['∆H (A-rich)'] <= 10) & (df['∆H (B-rich)'] <= 10)]
#df_nooutliers

In [11]:
def counter(dataframe, column):
    
    count = pd.value_counts(dataframe[column])
    count_df = count.to_frame()
    count_df['percent']=round(count_df[column]/sum(count)*100,2)
    
    print('Total entries: ', sum(count))
    
    return count_df

## PCA using ``pyOD``

- what DFT data is anomalous? Removing 10% of the data that appears anomalous based on the "target" values calculated from DFT, or the "descriptor" values.

In [12]:
def myPCA(dataframe, data_type, n_components=None, n_selected_components=None, contamination=0.1):
    array = np.array(dataframe[data_type])
    
    # using PCA model from pymod
    # model will identify ~ 10% of data as outliers
    clf = PCA(contamination=contamination, n_components=n_components, n_selected_components=n_selected_components)

    # fitting the data
    clf.fit(array)

    # classifying the targets as either inliers(0) or outliers(1)
    ypred = clf.predict(array)
    
    # df of outliers
    df_outlier = dataframe[ypred == 1]

    # df without outliers
    df_nooutlier = dataframe[ypred == 0]
    
    return df_outlier, df_nooutlier

## Using ``df``

### PCA with target vals

In [13]:
df_pca_out_tar, df_pca_in_tar = myPCA(df, output)

### PCA with descriptor vals

In [14]:
df_pca_out_des, df_pca_in_des = myPCA(df, descriptors)

### Similar outlier rows between the PCA models

In [15]:
def similar_df(df1, df2):
    df_comm = pd.concat([df1, df2])

    df_comm = df_comm.reset_index(drop=True)

    df_gpby = df_comm.groupby(list(df_comm.columns))

    idx = [x[0] for x in df_gpby.groups.values() if len(x) != 1]
    
    
    return df_comm.reindex(idx)

In [19]:
sim = similar_df(df_pca_out_tar,df_pca_out_des)
#sim

### Differences between dataframes

In [20]:
def diff_df(df1, df2):
    df_diff = pd.concat([df1,df2]).drop_duplicates(keep=False)
    
    return df_diff

In [21]:
diff = diff_df(df_pca_out_tar,df_pca_out_des)
#diff

In [22]:
counter(df_pca_out_tar, 'Type')

Total entries:  86


,Type,percent
IV-IV,63,73.26
III-V,16,18.60
II-VI,7,8.14


In [23]:
counter(df_pca_in_tar, 'Type')

Total entries:  767


,Type,percent
II-VI,499,65.06
III-V,139,18.12
IV-IV,129,16.82


In [24]:
counter(df_pca_out_des, 'Type')

Total entries:  86


,Type,percent
II-VI,45,52.33
III-V,22,25.58
IV-IV,19,22.09


In [25]:
counter(df_pca_in_des, 'Type')

Total entries:  767


,Type,percent
II-VI,461,60.10
IV-IV,173,22.56
III-V,133,17.34


## Using ``df_nooutliers``

In [26]:
dfnout_pca_out_tar, dfnout_pca_in_tar = myPCA(df_nooutliers, output)

In [27]:
counter(dfnout_pca_out_tar, 'Type')

Total entries:  77


,Type,percent
IV-IV,63,81.82
III-V,9,11.69
II-VI,5,6.49


In [28]:
dfnout_pca_out_des, dfnout_pca_in_des = myPCA(df_nooutliers, descriptors)

In [29]:
#dfnout_pca_in_des

In [30]:
#dfnout_pca_out_des

In [31]:
counter(dfnout_pca_out_des, 'Type')

Total entries:  77


,Type,percent
II-VI,41,53.25
III-V,20,25.97
IV-IV,16,20.78


----
---

## Comparing the effect of number of components on how PCA picks outliers
- the only real difference I saw was going down to 2 components

In [32]:
all_out, all_no = myPCA(df_nooutliers, descriptors)

In [33]:
twenty_out, twenty_no = myPCA(df_nooutliers, descriptors, n_components=20, n_selected_components=20)

In [31]:
#similar_df(all_out, twenty_out)

In [32]:
#diff_df(all_out, twenty_out)

---------------------
---------

## Evaluating RFR on data after PCA

In [28]:
def RFR_abbr(df, o_start=4, o_end=10):
    '''
    o_start: int. column index of target value. (4 is the beginning)
    o_end: int. column index of target value. (10 is the end)
    '''
    descriptors = df.columns[10:]
    output = df.columns[o_start:o_end]
    train,test = train_test_split(df,test_size=0.22, random_state=130)
    clf = RandomForestRegressor(n_jobs=2, random_state=130)
    
    frames_list = []
    train_rmse_list = [] 
    test_rmse_list = []
    
    for o in output:
        clf.fit(train[descriptors], train[o])

        trainpred = clf.predict(train[descriptors])
        testpred = clf.predict(test[descriptors])
        
        train_rmse = mean_squared_error(train[o],trainpred, squared=False)
        test_rmse = mean_squared_error(test[o],testpred, squared=False)
        
        train_rmse_list.append(train_rmse)
        test_rmse_list.append(test_rmse)

    
    d = {'output': output, 'train rmse': train_rmse_list, 'test rmse': test_rmse_list}
    
    rmse_df = pd.DataFrame(data=d)

    return rmse_df

In [29]:
def RFR_type(df, o_start=4, o_end=10):
    
    descriptors = df.columns[10:]
    output = df.columns[o_start:o_end]
    train,test = train_test_split(df,test_size=0.22, random_state=130)
    clf = RandomForestRegressor(n_jobs=2, random_state=130)
    
    train_26 = train[train['Type']=='II-VI']
    train_35 = train[train['Type']=='III-V']
    train_44 =  train[train['Type']=='IV-IV']

    test_26 = test[test['Type']=='II-VI']
    test_35 = test[test['Type']=='III-V']
    test_44 = test[test['Type']=='IV-IV']
    
    traintest_list = [(train_26, test_26),(train_35, test_35),(train_44, test_44)]
    
    tt_dict = {}
    
    for tt in traintest_list:
        
        key = str(tt[0].Type.unique())
        
        train_rmse_list = [] 
        test_rmse_list = []
        
        for o in output:
            clf.fit(train[descriptors], train[o])

            trainpred = clf.predict(tt[0][descriptors])
            testpred = clf.predict(tt[1][descriptors])

            train_rmse = mean_squared_error(tt[0][o],trainpred, squared=False)
            test_rmse = mean_squared_error(tt[1][o],testpred, squared=False)

            
            train_rmse_list.append(train_rmse)
            test_rmse_list.append(test_rmse)
        
        #print('train', train_rmse_list)
        #print('test', test_rmse_list)
            
        d = {'output': output, 'train rmse': train_rmse_list, 'test rmse': test_rmse_list}

        rmse_df = pd.DataFrame(data=d)
        
        tt_dict[key] = (rmse_df)
    
    return tt_dict

### PCA target

In [81]:
RFR_abbr(df_pca_in_tar)

,output,train rmse,test rmse
0,∆H (A-rich),0.521188,1.711316
1,∆H (B-rich),0.668668,1.969930
2,(+2/+1),0.180982,0.627352
3,(+1/0),0.187063,0.550364
4,(0/-1),0.170332,0.490597
5,(-1/-2),0.127100,0.344000


In [83]:
RFR_type(df_pca_in_tar)

{"['II-VI']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.398731   1.165730
 1  ∆H (B-rich)    0.532737   1.600196
 2      (+2/+1)    0.170987   0.421911
 3       (+1/0)    0.174467   0.430523
 4       (0/-1)    0.159486   0.408324
 5      (-1/-2)    0.122719   0.319528,
 "['III-V']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.742813   1.975805
 1  ∆H (B-rich)    0.960489   2.639330
 2      (+2/+1)    0.193675   0.749402
 3       (+1/0)    0.193055   0.584927
 4       (0/-1)    0.201011   0.646550
 5      (-1/-2)    0.119188   0.393656,
 "['IV-IV']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.645091   2.680194
 1  ∆H (B-rich)    0.761205   2.294364
 2      (+2/+1)    0.204237   0.972028
 3       (+1/0)    0.225625   0.808873
 4       (0/-1)    0.175561   0.560239
 5      (-1/-2)    0.150858   0.369650}

### PCA descriptors

In [84]:
RFR_abbr(df_pca_in_des)

,output,train rmse,test rmse
0,∆H (A-rich),0.690964,2.052092
1,∆H (B-rich),0.804967,2.739747
2,(+2/+1),0.215243,0.580923
3,(+1/0),0.204297,0.532751
4,(0/-1),0.163685,0.419417
5,(-1/-2),0.118267,0.340651


In [85]:
RFR_type(df_pca_in_des)

{"['II-VI']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.445371   1.025345
 1  ∆H (B-rich)    0.650307   1.494498
 2      (+2/+1)    0.181521   0.390340
 3       (+1/0)    0.174682   0.446580
 4       (0/-1)    0.151340   0.380506
 5      (-1/-2)    0.110798   0.324668,
 "['III-V']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.840807   2.217076
 1  ∆H (B-rich)    0.983819   2.706248
 2      (+2/+1)    0.251696   0.622941
 3       (+1/0)    0.288901   0.491263
 4       (0/-1)    0.208790   0.451170
 5      (-1/-2)    0.125104   0.274852,
 "['IV-IV']":         output  train rmse  test rmse
 0  ∆H (A-rich)    1.057928   3.114466
 1  ∆H (B-rich)    1.028863   4.197471
 2      (+2/+1)    0.268930   0.807760
 3       (+1/0)    0.204346   0.688082
 4       (0/-1)    0.157925   0.464156
 5      (-1/-2)    0.133105   0.405730}

### df_nooutliers (formation energy values > 10 eV removed)

In [86]:
RFR_abbr(dfnout_pca_in_tar)

,output,train rmse,test rmse
0,∆H (A-rich),0.453549,1.239188
1,∆H (B-rich),0.524834,1.490647
2,(+2/+1),0.177215,0.509280
3,(+1/0),0.201798,0.488115
4,(0/-1),0.164993,0.451035
5,(-1/-2),0.119997,0.329110


In [87]:
RFR_type(dfnout_pca_in_tar)

{"['II-VI']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.351041   0.937208
 1  ∆H (B-rich)    0.439929   1.313003
 2      (+2/+1)    0.166860   0.438792
 3       (+1/0)    0.178266   0.407402
 4       (0/-1)    0.152526   0.463214
 5      (-1/-2)    0.116928   0.332916,
 "['III-V']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.612832   2.080744
 1  ∆H (B-rich)    0.739946   2.172493
 2      (+2/+1)    0.198038   0.711526
 3       (+1/0)    0.212959   0.690511
 4       (0/-1)    0.186090   0.443661
 5      (-1/-2)    0.127007   0.298336,
 "['IV-IV']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.659473   1.071020
 1  ∆H (B-rich)    0.596540   1.231551
 2      (+2/+1)    0.201566   0.509922
 3       (+1/0)    0.291917   0.519368
 4       (0/-1)    0.198201   0.407455
 5      (-1/-2)    0.126832   0.345094}

In [88]:
RFR_abbr(dfnout_pca_in_des)

,output,train rmse,test rmse
0,∆H (A-rich),0.483860,1.360507
1,∆H (B-rich),0.543609,1.492331
2,(+2/+1),0.197292,0.713207
3,(+1/0),0.194802,0.487206
4,(0/-1),0.167198,0.455665
5,(-1/-2),0.117659,0.322446


In [89]:
RFR_type(dfnout_pca_in_des)

{"['II-VI']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.377965   1.107437
 1  ∆H (B-rich)    0.450525   1.388783
 2      (+2/+1)    0.169078   0.509091
 3       (+1/0)    0.174737   0.486724
 4       (0/-1)    0.160910   0.467613
 5      (-1/-2)    0.115325   0.332616,
 "['III-V']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.684610   1.770361
 1  ∆H (B-rich)    0.798284   1.812151
 2      (+2/+1)    0.260531   0.443706
 3       (+1/0)    0.237212   0.453829
 4       (0/-1)    0.193921   0.449996
 5      (-1/-2)    0.129856   0.312493,
 "['IV-IV']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.594868   1.582138
 1  ∆H (B-rich)    0.574017   1.496213
 2      (+2/+1)    0.224681   1.151608
 3       (+1/0)    0.219765   0.509642
 4       (0/-1)    0.164330   0.429263
 5      (-1/-2)    0.114858   0.303376}